<a href="https://colab.research.google.com/github/hyesungKomet/rokaf_ai/blob/main/Elice_2_2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential

def build_model1(input_shape):
    model = layers.Conv2D(1, kernel_size = (3,3), strides=(1,1), padding="same", #same은 입력 이미지 크기와 같게 출력해준다(그렇게 알아서 패딩해줌)
                          activation="relu", 
                          input_shape=input_shape[1:]) #맨 처음 layer이면 입력이미지의 사이즈를 알려줘야 함!
    
    return model

def build_model2(input_shape): #층 쌓기 방법 1
    model = Sequential([layers.Conv2D(4, kernel_size = (3,3), strides=(1,1), padding="same", input_shape=input_shape[1:]),
                        layers.Conv2D(4, kernel_size=(3,3), strides=(1,1), padding="same")])
    
    return model

def build_model3(input_shape): #층 쌓기 방법 2
    model = Sequential()
    
    model.add(layers.Conv2D(2, kernel_size=(3,3), strides=(1,1), padding="same", input_shape=input_shape[1:]))
    model.add(layers.Conv2D(4, kernel_size=(3,3), strides=(1,1), padding="same"))
    model.add(layers.Conv2D(8, kernel_size=(3,3), strides=(1,1)))
    
    return model

def main():
    input_shape = (1, 5, 5, 1)
    
    model1 = build_model1(input_shape)
    model2 = build_model2(input_shape)
    model3 = build_model3(input_shape)

    x = tf.ones(input_shape)
    print("model1을 통과한 결과:", model1(x).shape)
    print("model2을 통과한 결과:", model2(x).shape)
    print("model3을 통과한 결과:", model3(x).shape)

if __name__=="__main__":
    main()

model1을 통과한 결과: (1, 5, 5, 1)
model2을 통과한 결과: (1, 5, 5, 4)
model3을 통과한 결과: (1, 3, 3, 8)


In [ ]:
import os

import tensorflow as tf
from tensorflow.keras import layers, Sequential, Input
from tensorflow.keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt

SEED = 2021

def load_cifar10_dataset():
    train_X = np.load("./dataset/cifar10_train_X.npy")
    train_y = np.load("./dataset/cifar10_train_y.npy")
    test_X = np.load("./dataset/cifar10_test_X.npy")
    test_y = np.load("./dataset/cifar10_test_y.npy")
    
    # 정규화(0~1 사이의 값으로)
    train_X, test_X = train_X / 255.0, test_X / 255.0
    
    return train_X, train_y, test_X, test_y

def build_mlp_model(img_shape, num_classes=10):
    model = Sequential()

    model.add(Input(shape=img_shape))
    
    # 2차원 이미지를 1차원으로 변형
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation="relu"))
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(num_classes, activation="softmax"))

    return model
    
def plot_history(hist):
    train_loss = hist.history["loss"]
    train_acc = hist.history["accuracy"]
    valid_loss = hist.history["val_loss"]
    valid_acc = hist.history["val_accuracy"]
    
    fig = plt.figure(figsize=(8, 6))
    plt.plot(train_loss)
    plt.plot(valid_loss)
    plt.title('Loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Train', 'Valid'], loc='upper right')
    plt.savefig("loss.png")
    
    fig = plt.figure(figsize=(8, 6))
    plt.plot(train_acc)
    plt.plot(valid_acc)
    plt.title('Accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.savefig("accuracy.png")

def main(model=None, epochs=10):
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    
    train_X, train_y, test_X, test_y = load_cifar10_dataset()
    img_shape = train_X[0].shape
    
    # Adam optimizer
    optimizer = Adam(learning_rate=0.001)

    mlp_model = model
    if model is None:
        mlp_model = build_mlp_model(img_shape)
    
    # 모델의 optimizer, 손실 함수, 평가 지표 설정
    mlp_model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    # 모델 학습을 위한 hyperparameter
    hist = mlp_model.fit(train_X, train_y, epochs=epochs, batch_size=64, validation_split=0.2, shuffle=True, verbose=2)
    
    plot_history(hist)
    test_loss, test_acc = mlp_model.evaluate(test_X, test_y)
    print("Test Loss: {:.5f}, Test Accuracy: {:.3f}%".format(test_loss, test_acc * 100))
    
    return optimizer, hist

if __name__ == "__main__":
    main()

## MLP와 CNN 모델 비교

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential, Input
from tensorflow.keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt

SEED = 2021

def load_cifar10_dataset():
    train_X = np.load("./dataset/cifar10_train_X.npy")
    train_y = np.load("./dataset/cifar10_train_y.npy")
    test_X = np.load("./dataset/cifar10_test_X.npy")
    test_y = np.load("./dataset/cifar10_test_y.npy")
    
    train_X, test_X = train_X / 255.0, test_X / 255.0
    
    return train_X, train_y, test_X, test_y
    
def build_mlp_model(img_shape, num_classes=10):
    model = Sequential()

    model.add(Input(shape=img_shape))
    
    # MLP 모델
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation="relu"))
    model.add(layers.Dense(1024, activation="relu"))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(num_classes, activation="softmax"))


    return model

def build_cnn_model(img_shape, num_classes=10):
    model = Sequential()

    # CNN 모델
    model.add(layers.Conv2D(16, kernel_size=(3,3), padding="same", activation="relu", input_shape=img_shape))
    model.add(layers.Conv2D(32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.MaxPool2D(2))
    model.add(layers.Conv2D(64, kernel_size=(3,3), padding="same", strides=(2,2), activation="relu"))
    model.add(layers.Conv2D(64, kernel_size=(3,3), padding="same", strides=(2,2), activation="relu"))
    model.add(layers.MaxPool2D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation="relu"))
    model.add(layers.Dense(num_classes, activation="softmax"))

    return model
    
def plot_history(hist):
    train_loss = hist.history["loss"]
    train_acc = hist.history["accuracy"]
    valid_loss = hist.history["val_loss"]
    valid_acc = hist.history["val_accuracy"]
    
    fig = plt.figure(figsize=(8, 6))
    plt.plot(train_loss)
    plt.plot(valid_loss)
    plt.title('Loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Train', 'Valid'], loc='upper right')
    plt.savefig("loss.png")
    
    
    fig = plt.figure(figsize=(8, 6))
    plt.plot(train_acc)
    plt.plot(valid_acc)
    plt.title('Accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.savefig("accuracy.png")
    
    
def run_model(model, train_X, train_y, test_X, test_y, epochs=10):
    # Adam optimizer
    optimizer = Adam(learning_rate=0.001)
    model.summary()
    # 모델의 optimizer, 손실 함수, 평가 지표 설정
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    
    # 모델 학습을 위한 hyperparameter 설정
    hist = model.fit(train_X, train_y, epochs=epochs, batch_size=64, validation_split=0.2, shuffle=True, verbose=2)
    
    plot_history(hist)
    test_loss, test_acc = model.evaluate(test_X, test_y)
    print("Test Loss: {:.5f}, Test Accuracy: {:.3f}%".format(test_loss, test_acc * 100))
    
    return optimizer, hist

def model_summary(model):
    model.summary()

def main():
    tf.random.set_seed(SEED)
    np.random.seed(SEED)
    
    #train_X, train_y, test_X, test_y = load_cifar10_dataset()
    #img_shape = train_X[0].shape
    img_shape = (224, 224, 3)
    mlp_model = build_mlp_model(img_shape)
    cnn_model = build_cnn_model(img_shape)

    
    print("=" * 30, "MLP 모델", "=" * 30)
    #run_model(mlp_model, train_X, train_y, test_X, test_y)
    model_summary(mlp_model)
    
    print()
    print("=" * 30, "CNN 모델", "=" * 30)
    #run_model(cnn_model, train_X, train_y, test_X, test_y)
    model_summary(cnn_model)

if __name__ == "__main__":
    main()

============================== MLP 모델 ==============================
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 150528)            0         
                                                                 
 dense_4 (Dense)             (None, 4096)              616566784 
                                                                 
 dense_5 (Dense)             (None, 1024)              4195328   
                                                                 
 dense_6 (Dense)             (None, 256)               262400    
                                                                 
 dense_7 (Dense)             (None, 64)                16448     
                                                                 
 dense_8 (Dense)             (None, 10)                650       
                                                   

## VGG16 모델 구현
- 2014년 ImageNet Challenge에서 2위 차지한 모델(1위는 GoogleNet)
- but 1위보다 훨씬 단순한 구조로 많이 사용됐음
- 모든 커널의 크기를 (3,3)으로 맞춰서 본격적으로 layer를 깊게 쌓기 시작한 모델
- AlexNet보다 2배 이상 늘어난 16 또는 19개의 layer로 이루어짐

In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers

def build_vgg16():
    # Sequential 모델 선언
    model = Sequential()
    
    # 첫번째 Block
    model.add(layers.Conv2D(64, kernel_size=(3,3), padding="same", activation="relu", input_shape=(224, 224, 3)))
    model.add(layers.Conv2D(64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.MaxPooling2D(2))
    
    # 두번째 Block
    model.add(layers.Conv2D(128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.Conv2D(128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.MaxPooling2D(2))
    
    # 세번째 Block
    model.add(layers.Conv2D(256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.Conv2D(256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.Conv2D(256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.MaxPooling2D(2))
    
    # 네번째 Block
    model.add(layers.Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.MaxPooling2D(2))
    
    # 다섯번째 Block
    model.add(layers.Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.Conv2D(512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(layers.MaxPooling2D(2))
    
    # Fully Connected Layer
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation="relu"))
    model.add(layers.Dense(4096, activation="relu"))
    model.add(layers.Dense(1000, activation="softmax"))
    
    return model

def main():
    model = build_vgg16()
    model.summary()
    
if __name__ == "__main__":
    main()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

## ResNet 구현
![](https://images.velog.io/images/junyoung9696/post/3137e50c-b52f-4cdd-8ae8-2faf497efe84/r10.png)

* Residual Connection (== Skip Connection)  
Layer가 많은 경우 발생하는 Gradient Vanishing(기울기 소실) 문제를 해결!

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequential

class ResidualBlock(Model):
    def __init__(self, num_kernels, kernel_size):
        super(ResidualBlock, self).__init__()

        # 2개의 Conv2D Layer
        self.conv1 = layers.Conv2D(num_kernels, kernel_size=kernel_size, padding="same", activation="relu")
        self.conv2 = layers.Conv2D(num_kernels, kernel_size=kernel_size, padding="same", activation="relu")
        
        self.relu = layers.Activation("relu")
        
        # Add Layer 추가 - ResNet의 핵심!!!
        self.add = layers.Add() # x, y 텐서를 (x, y)로 묶어주는 역할 - layer 관리 편하기 위함

    def call(self, input_tensor): #그림 이해!!
        x = self.conv1(input_tensor)
        x = self.conv2(x)

        x = self.add([x, input_tensor]) # 두 값을 더하는 과정(f(x)와 x를 더함)
        x = self.relu(x)
        
        return x
        
def build_resnet(input_shape, num_classes):
    model = Sequential()
    
    model.add(layers.Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu", input_shape=input_shape))
    model.add(layers.MaxPool2D(2))
    
    model.add(ResidualBlock(64, (3, 3)))
    model.add(ResidualBlock(64, (3, 3)))
    model.add(ResidualBlock(64, (3, 3)))
    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(num_classes, activation="softmax"))
    
    return model
    
def main():
    input_shape = (32, 32, 3)
    num_classes = 10

    model = build_resnet(input_shape, num_classes)
    model.summary()

if __name__=="__main__":
    main()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 32, 32, 64)        1792      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 16, 16, 64)       0         
 2D)                                                             
                                                                 
 residual_block (ResidualBlo  (None, 16, 16, 64)       73856     
 ck)                                                             
                                                                 
 residual_block_1 (ResidualB  (None, 16, 16, 64)       73856     
 lock)                                                           
                                                                 
 residual_block_2 (ResidualB  (None, 16, 16, 64)       73856     
 lock)                                                